<a href="https://colab.research.google.com/github/masdesouza/study_artificial_intelligence/blob/main/ProjetoDeepLearningWithRelu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Apr 28 18:12:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD, Adamax, RMSprop, Adam

from keras.utils import to_categorical

import numpy as np


batch_size =  128
num_classes = 10
epochs = 120

# input image dimensions
img_rows, img_cols = 32, 32

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#input_shape = (3,32,32)
input_shape=(32, 32, 3)

#Conversão de inteiros para real
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#Normalização para o range 0-1
x_train /= 255
x_test /= 255


print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

#x_train=np.rollaxis(x_train, 3, 1)
#x_test=np.rollaxis(x_test, 3, 1)


#Construindo o modelo
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))

model.add(Flatten())
model.add(Dense(700, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))



#Compilando o modelo
#opt = SGD(lr=0.001, momentum=0.9)
#opt = Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
opt = Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07) #beta e epsilon normalmente não se ajusta, só a taxa

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])



#Avaliando o modelo com a base de teste

# fit model
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), verbose=1)


# evaluate model
#_, acc = model.evaluate(x_test, y_test, verbose=0)
#print('> %.3f' % (acc * 100.0))



score = model.evaluate(x_test, y_test, verbose=0)


print('Test loss:', ' %.3f' %  ( score[0] ) )
print('Test accuracy:', ' %.3f' % ( score[1]* 100.0) )

170500096/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Epoch 1/120
391/391 [==============================] - 26s 25ms/step - loss: 2.0933 - accuracy: 0.2285 - val_loss: 1.4746 - val_accuracy: 0.4615
Epoch 2/120
391/391 [==============================] - 9s 22ms/step - loss: 1.4431 - accuracy: 0.4704 - val_loss: 1.2221 - val_accuracy: 0.5540
Epoch 3/120
391/391 [==============================] - 9s 22ms/step - loss: 1.2133 - accuracy: 0.5605 - val_loss: 1.0415 - val_accuracy: 0.6241
Epoch 4/120
391/391 [==============================] - 9s 22ms/step - loss: 1.0365 - accuracy: 0.6290 - val_loss: 0.9040 - val_accuracy: 0.6772
Epoch 5/120
391/391 [==============================] - 9s 22ms/step - loss: 0.9068 - accuracy: 0.6767 - val_loss: 0.8622 - val_accuracy: 0.6998
Epoch 6/120
391/391 [==============================] - 9s 22ms/step - loss: 0.7926 - accuracy: 0.7197 - val_loss: 0.7484 - val_accuracy: 0.